# Notebook for testing mutlimodal capability of Gemma

## Imports 
*Necessary for running notebook*

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it", device_map="auto")

### Testing out gemma instruct on text generation

In [ ]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_length=40)
print(tokenizer.decode(outputs[0]))